# Imports

In [ ]:
import pickle
import pandas as pd
from p05_nlp import display_topics
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label, IntSlider, FloatSlider
from ipywidgets import Button, Layout

In [ ]:
from PIL import Image
import requests
from io import BytesIO

# Pickles

In [ ]:
with open("df_all_cleaned.pickle", "rb") as read_file:
    df_all = pickle.load(read_file)

In [ ]:
with open("df_topics.pickle", "rb") as read_file:
    df_topics = pickle.load(read_file)

In [ ]:
with open("reviews_nmf_df.pickle", "rb") as read_file:
    reviews_nmf_df = pickle.load(read_file)

In [ ]:
with open("nmf_topics.pickle", "rb") as read_file:
    topic_labels = pickle.load(read_file)

In [ ]:
with open("nmf_components.pickle", "rb") as read_file:
    nmf_components = pickle.load(read_file)

In [ ]:
with open("reviews_feature_names_nmf.pickle", "rb") as read_file:
    topic_contents = pickle.load(read_file)

In [ ]:
display_topics(nmf_components, topic_contents, num_top_words=15, topic_names=topic_labels)

# Functions

Generic/straightforward recommendation system

In [ ]:
def recommender(movie):
    print("Movies most similar to '{}':".format(movie))
    movie_mask = df_topics['title']== movie
    movie_sim = cosine_similarity(reviews_nmf_df[~movie_mask],reviews_nmf_df[movie_mask])
    movie_sim_df = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_sim_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    return movie_sim_df.sort_values(by='similarity',ascending=False).head(5)

Function that takes topic distribution as a dictionary

In [ ]:
dict_in = {'Gory/Scary': [0.012183], "Silly": [0.41586], "Living Dead": [0.909248]}

In [ ]:
title_mask = df_topics['title']=='Shaun of the Dead'
dict_in = df_topics[title_mask].iloc[:,1:].to_dict('list') 

In [ ]:
def topic_levels(dict_in):
    '''
    Generates a list of top 5 most similar movies based on topic distribution from sliders.
    Does not include the input movie used to generate sliders.
    'Similarity' column indicates similarity to the sliders, not to the input movie.
    '''
    # create df_in from dict_in
    df_in = pd.DataFrame.from_dict(dict_in)
    topics = df_in.columns
    # get slice of df_topics
    df_slice = df_topics.loc[~title_mask,topics]
    # do cosine similarity
    movie_sim = cosine_similarity(df_slice,df_in)
    # get recommendations
    movie_rec = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_rec.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    #return list of movies
    return movie_rec.sort_values(by='similarity',ascending=False).head(5)

In [ ]:
topic_levels(dict_in)

# Full Recommendation Widget

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label, IntSlider
from ipywidgets import Button, Layout

In [ ]:
dict_in

## All in one (doesn't quite work)

In [ ]:
btn_layout = Layout(width='20%', height='29px')
gen_btn = Button(description='Generate', layout=btn_layout)
rec_btn = Button(description='Recommend', layout=btn_layout)

text_layout = Layout(width='50%')
movie_title = widgets.Text(placeholder='Enter a movie title', description='Title:', 
                           layout=text_layout, disabled=False)

display(HBox([movie_title, gen_btn]))

def on_gen_btn_clicked(gen_btn):
    movie_info(movie_title)
    dict_in = df_topics[df_topics['title']==movie_title.value].iloc[:,1:].to_dict('list')
    generate_sliders(movie_title, dict_in)
    # display the second button "recommend movies based on these topic levels"
    display(rec_btn)

def movie_info(movie_title):
    title = movie_title.value
    print("You entered: {}".format(title))
    url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    display(img)
        
def generate_sliders(movie_title, dict_in):
    sliders={}
    float_text={}
    style = {'description_width': '150px'}
    sliders_layout = {'width': '60%'}
    float_text_layout = {'width': '70px'}
    for key, val in dict_in.items():
        value = val[0]*100
        sliders[key] = widgets.FloatSlider(value=value, min=0.0, max=100.0, 
                                           readout_format='.2f', description=key,
                                           style=style, layout=sliders_layout)
        float_text[key] = widgets.FloatText(value=value, step='0.01', layout=float_text_layout)
        display(HBox([sliders[key], float_text[key]]))

    links = {}
    for key, val in dict_in.items():
        links[key] = widgets.jslink((float_text[key], 'value'), (sliders[key], 'value'))
        
def on_rec_btn_clicked(rec_btn):
    # display recommended movies based on topic distribution sliders
    pass

def recommend(dict_in):
    sliders_dict = {}
    topics = list(dict_in.keys())
    for top in topics:
        sliders_dict[top] = [sliders[top].value/100]
    recommended = topic_levels(sliders_dict)

def topic_levels(sliders_dict):
    '''
    Generates a list of top 5 most similar movies based on topic distribution from sliders.
    Does not include the input movie used to generate sliders.
    'Similarity' column indicates similarity to the sliders, not to the input movie.
    '''
    # create df_in from sliders_dict
    df_in = pd.DataFrame.from_dict(sliders_dict)
    topics = df_in.columns
    # get slice of df_topics
    df_slice = df_topics.loc[~title_mask,topics]
    # do cosine similarity
    movie_sim = cosine_similarity(df_slice,df_in)
    # get recommendations
    movie_rec = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_rec.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    #return list of movies
    return movie_rec.sort_values(by='similarity',ascending=False).head(5)

gen_btn.on_click(on_gen_btn_clicked)

## All in one, version 2 (works?)

In [ ]:
btn_layout = Layout(width='20%', height='29px')
gen_btn = Button(description='Generate', layout=btn_layout)
rec_btn = Button(description='Recommend', layout=btn_layout)

text_layout = Layout(width='50%')
movie_title = widgets.Text(placeholder='Enter a movie title', description='Title:', 
                           layout=text_layout, disabled=False)

display(HBox([movie_title, gen_btn]))
    
def on_gen_btn_clicked(gen_btn):
    movie_info(movie_title)
    sliders={}
    float_text={}
    sliders_style = {'description_width': '150px'}
    sliders_layout = {'width': '60%'}
    float_text_layout = {'width': '70px'}
    dict_in = df_topics[df_topics['title']==movie_title.value].iloc[:,1:].to_dict('list')
    
    for key, val in dict_in.items():
        value = val[0]*100
        sliders[key] = widgets.FloatSlider(value=value, min=0.0, max=100.0, 
                                           readout_format='.2f', description=key,
                                           style=sliders_style, layout=sliders_layout)
        float_text[key] = widgets.FloatText(value=value, step='0.01', layout=float_text_layout)
    
    links = {}
    for key, val in dict_in.items():
        links[key] = widgets.jslink((float_text[key], 'value'), (sliders[key], 'value'))
        display(HBox([sliders[key], float_text[key]]))        
    
    # display the second button "recommend movies based on these topic levels"
    display(rec_btn)

def movie_info(movie_title):
    title = movie_title.value
    print("You entered: {}".format(title))
    url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    display(img)
        
def on_rec_btn_clicked(rec_btn):
    # display recommended movies based on topic distribution sliders
    dict_in = df_topics[df_topics['title']==movie_title.value].iloc[:,1:].to_dict('list')
    sliders_dict = {}
    topics = list(dict_in.keys())
    for top in topics:
        sliders_dict[top] = [sliders[top].value/100]
    recommended = topic_levels(sliders_dict)
    titles = list(recommended['title'])
    show_posters(titles)
    

def show_posters(titles):
    posters = {}
    for title in titles:
        url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)) # to save img as variable
        posters[title] = img
        display(img)
#     images = [img for title,img in posters.items()]

def topic_levels(sliders_dict):
    '''
    Generates a list of top 5 most similar movies based on topic distribution from sliders.
    Does not include the input movie used to generate sliders.
    'Similarity' column indicates similarity to the sliders, not to the input movie.
    '''
    # create df_in from sliders_dict
    df_in = pd.DataFrame.from_dict(sliders_dict)
    topics = df_in.columns
    # get slice of df_topics
    df_slice = df_topics.loc[~title_mask,topics]
    # do cosine similarity
    movie_sim = cosine_similarity(df_slice,df_in)
    # get recommendations
    movie_rec = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_rec.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    #return list of movies
    return movie_rec.sort_values(by='similarity',ascending=False).head(5)

gen_btn.on_click(on_gen_btn_clicked)
rec_btn.on_click(on_rec_btn_clicked)

## Interactive Recommendation Widget

In [ ]:
# create text box to receive input movie
btn_layout = Layout(width='20%', height='29px')
gen_btn = Button(description='Generate', layout=btn_layout)
# rec_btn = Button(description='Recommend', layout=btn_layout)

text_layout = Layout(width='50%')
movie_title = widgets.Text(placeholder='Enter a movie title', description='Title:', 
                           layout=text_layout, disabled=False)
display(HBox([movie_title, gen_btn]))

def movie_info(movie_title):
    title = movie_title.value
    print("You entered: {}".format(title))
    url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    display(img)
    
def on_gen_btn_clicked(gen_btn):
    movie_info(movie_title)
    
gen_btn.on_click(on_gen_btn_clicked)


In [ ]:
# generate sliders based on input movie's topic distribution dictionary
title_mask = df_topics['title']==movie_title.value
dict_in = df_topics[title_mask].iloc[:,1:].to_dict('list') 
sliders={}
float_text={}
style = {'description_width': '150px'}
sliders_layout = {'width': '60%'}
float_text_layout = {'width': '70px'}
for key, val in dict_in.items():
    value = val[0]*100
    sliders[key] = widgets.FloatSlider(value=value, min=0.0, max=100.0, 
                                       readout_format='.2f', description=key,
                                       style=style, layout=sliders_layout)
    float_text[key] = widgets.FloatText(value=value, step='0.01', layout=float_text_layout)
    display(HBox([sliders[key], float_text[key]]))

links = {}
for key, val in dict_in.items():
    links[key] = widgets.jslink((float_text[key], 'value'), (sliders[key], 'value'))

In [ ]:
# get recommended movies
sliders_dict = {}
topics = list(dict_in.keys())
for top in topics:
    sliders_dict[top] = [sliders[top].value/100]

def topic_levels(dict_in):
    '''
    Generates a list of top 5 most similar movies based on topic distribution from sliders.
    Does not include the input movie used to generate sliders.
    'Similarity' column indicates similarity to the sliders, not to the input movie.
    '''
    # create df_in from dict_in
    df_in = pd.DataFrame.from_dict(dict_in)
    topics = df_in.columns
    # get slice of df_topics
    df_slice = df_topics.loc[~title_mask,topics]
    # do cosine similarity
    movie_sim = cosine_similarity(df_slice,df_in)
    # get recommendations
    movie_rec = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_rec.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    #return list of movies
    return movie_rec.sort_values(by='similarity',ascending=False).head(5)
recommended = topic_levels(sliders_dict)
recommended

In [ ]:
# show the posters for recommended movies
titles = list(recommended['title'])
def show_posters(titles):
    posters = {}
    for title in titles:
        url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)) # to save img as variable
        posters[title] = img
        display(img)
#     print([img for title,img in posters.items()])
#     display(HBox[img for title,img in posters.items()])
    images = [img for title,img in posters.items()]
        
show_posters(titles)

# Appendix

Get the movie poster from the title

In [ ]:
from PIL import Image
import requests
from io import BytesIO

title='Shaun of the Dead'
url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
response = requests.get(url)
# img = Image.open(BytesIO(response.content)) # to save img as variable
Image.open(BytesIO(response.content))

In [ ]:
# titles=['Shaun of the Dead']
titles = list(recommended['title'])
def show_posters(titles):
    for title in titles:
        url = 'https://' + list(df_all[df_all['title']==title]['poster_url'])[0]
        response = requests.get(url)
        # img = Image.open(BytesIO(response.content)) # to save img as variable
        display(Image.open(BytesIO(response.content)))
        
show_posters(titles)

FloatSlider parameters

In [ ]:
widgets.FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

Text Box next to Slider; takes Float input and updates slider.

In [ ]:
text = widgets.FloatText()
slider = widgets.FloatSlider()
display(HBox([text,slider]))

mylink = widgets.jslink((text, 'value'), (slider, 'value'))

In [ ]:
text.keys

In [ ]:
btn_test = widgets.Button(description='Test')
display(btn_test)
 
 
def my_event_handler(btn_object):
    print('You pressed the {} button!'.format(btn_object.description))
 
btn_test.on_click(my_event_handler)

In [ ]:
rec_btn = Button(description='Recommend', layout=btn_layout)
display(rec_btn)